In [2]:
#libraries import
import sqlite3
import pandas as pd

In [3]:
#creating DB and tables
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE cryptocurrencies (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    price REAL NOT NULL
)
""")

cursor.execute("""
CREATE TABLE users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL
)
""")

cursor.execute("""
CREATE TABLE transactions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER,
    cryptocurrency_id INTEGER,
    amount REAL NOT NULL,
    FOREIGN KEY(user_id) REFERENCES users(id),
    FOREIGN KEY(cryptocurrency_id) REFERENCES cryptocurrencies(id)
)
""")



In [4]:
cryptos = [
    ("Bitcoin", 65000),
    ("Ethereum", 3300),
    ("Solana", 140),
    ("Cardano", 0.6),
    ("Polkadot", 7)
]

users = [
    ("Alice",),
    ("Bob",),
    ("Charlie",),
    ("Diana",),
    ("Eve",)
]

transactions = [
    (1, 1, 0.01),
    (1, 2, 1.5),
    (2, 1, 0.05),
    (2, 3, 20),
    (3, 4, 500),
    (4, 1, 0.2),
    (5, 2, 3),
    (3, 2, 2),
    (4, 5, 10),
    (5, 3, 5)
]

cursor.executemany("INSERT INTO cryptocurrencies (name, price) VALUES (?, ?)", cryptos)
cursor.executemany("INSERT INTO users (name) VALUES (?)", users)
cursor.executemany("INSERT INTO transactions (user_id, cryptocurrency_id, amount) VALUES (?, ?, ?)", transactions)

conn.commit()

# Вивід усіх таблиць
print("Cryptocurrencies:")
display(pd.read_sql("SELECT * FROM cryptocurrencies", conn))

print("Users:")
display(pd.read_sql("SELECT * FROM users", conn))

print("Transactions:")
display(pd.read_sql("SELECT * FROM transactions", conn))

Cryptocurrencies:


,id,name,price
0,1,Bitcoin,65000.0
1,2,Ethereum,3300.0
2,3,Solana,140.0
3,4,Cardano,0.6
4,5,Polkadot,7.0


Users:


,id,name
0,1,Alice
1,2,Bob
2,3,Charlie
3,4,Diana
4,5,Eve


Transactions:


,id,user_id,cryptocurrency_id,amount
0,1,1,1,0.01
1,2,1,2,1.50
2,3,2,1,0.05
3,4,2,3,20.00
4,5,3,4,500.00
5,6,4,1,0.20
6,7,5,2,3.00
7,8,3,2,2.00
8,9,4,5,10.00
9,10,5,3,5.00


In [5]:
df_all = pd.read_sql("SELECT * FROM transactions", conn)
df_all.to_csv("all_transactions.csv", index=False)
df_all

,id,user_id,cryptocurrency_id,amount
0,1,1,1,0.01
1,2,1,2,1.50
2,3,2,1,0.05
3,4,2,3,20.00
4,5,3,4,500.00
5,6,4,1,0.20
6,7,5,2,3.00
7,8,3,2,2.00
8,9,4,5,10.00
9,10,5,3,5.00


In [6]:
df_where = pd.read_sql("""
SELECT t.id, u.name AS user_name, c.name AS crypto, t.amount
FROM transactions t
JOIN users u ON t.user_id = u.id
JOIN cryptocurrencies c ON t.cryptocurrency_id = c.id
WHERE c.name = 'Bitcoin'
""", conn)
df_where.to_csv("bitcoin_transactions.csv", index=False)
df_where


,id,user_name,crypto,amount
0,1,Alice,Bitcoin,0.01
1,3,Bob,Bitcoin,0.05
2,6,Diana,Bitcoin,0.20


In [7]:
df_order = pd.read_sql("""
SELECT * FROM cryptocurrencies
ORDER BY price DESC
""", conn)
df_order.to_csv("cryptos_ordered.csv", index=False)
df_order


,id,name,price
0,1,Bitcoin,65000.0
1,2,Ethereum,3300.0
2,3,Solana,140.0
3,5,Polkadot,7.0
4,4,Cardano,0.6


In [8]:
df_group = pd.read_sql("""
SELECT c.name, SUM(t.amount) AS total_amount
FROM transactions t
JOIN cryptocurrencies c ON t.cryptocurrency_id = c.id
GROUP BY c.name
""", conn)
df_group.to_csv("group_by_crypto.csv", index=False)
df_group


,name,total_amount
0,Bitcoin,0.26
1,Cardano,500.00
2,Ethereum,6.50
3,Polkadot,10.00
4,Solana,25.00


In [9]:
df_having = pd.read_sql("""
SELECT c.name, SUM(t.amount) AS total_amount
FROM transactions t
JOIN cryptocurrencies c ON t.cryptocurrency_id = c.id
GROUP BY c.name
HAVING total_amount > 5
""", conn)
df_having.to_csv("having_gt_5.csv", index=False)
df_having


,name,total_amount
0,Cardano,500.0
1,Ethereum,6.5
2,Polkadot,10.0
3,Solana,25.0


In [10]:
df_join = pd.read_sql("""
SELECT t.id, u.name AS user, c.name AS cryptocurrency, t.amount, c.price, t.amount * c.price AS total_usd
FROM transactions t
JOIN users u ON t.user_id = u.id
JOIN cryptocurrencies c ON t.cryptocurrency_id = c.id
""", conn)
df_join.to_csv("joined_transactions.csv", index=False)
df_join

,id,user,cryptocurrency,amount,price,total_usd
0,1,Alice,Bitcoin,0.01,65000.0,650.0
1,2,Alice,Ethereum,1.50,3300.0,4950.0
2,3,Bob,Bitcoin,0.05,65000.0,3250.0
3,4,Bob,Solana,20.00,140.0,2800.0
4,5,Charlie,Cardano,500.00,0.6,300.0
5,6,Diana,Bitcoin,0.20,65000.0,13000.0
6,7,Eve,Ethereum,3.00,3300.0,9900.0
7,8,Charlie,Ethereum,2.00,3300.0,6600.0
8,9,Diana,Polkadot,10.00,7.0,70.0
9,10,Eve,Solana,5.00,140.0,700.0


In [11]:
conn.close()